### 이번 과제는 Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류해보는 과제입니다. clone coding을 하시되, 코드 주석을 line by line으로 꼼꼼하게 달아보시며 공부해보세요!

로컬에서 돌리셔도 되지만, colab에서 GPU로 돌려보는 것을 권장합니다!

## 데이터 로드 및 탐색

In [2]:
%%capture
!pip install transformers

In [3]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('drive/MyDrive/bbc-text.csv') # bbc-text.csv 파일 경로

In [6]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [7]:
print(len(df))

2225


In [8]:
df.groupby('category').count() #카테고리별 기사 개수

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


## BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다. 여러 종류를 시도해보세요.

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased') #pretrain된 BERT의 BertTokenizer 갖고 오기 (bert-base-cased 이므로 대소문자를 구분)
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          } #카테고리 딕셔너리화

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Dataset

In [10]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [labels[label] for label in df['category']] #라벨 -> 숫자로 인덱싱
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']] #토큰화 + 최대길이 지정, 패딩, 텐서변환

    def classes(self): #라벨리스트 반환
        return self.labels

    def __len__(self): #길이
        return len(self.labels)

    def get_batch_labels(self, idx): #특정 인덱스의 라벨
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx): #특정 인덱스의 토큰화결과
        return self.texts[idx]

    def __getitem__(self, idx): #라벨, 토큰화 결과 둘다 반환

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

## Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

In [17]:
class BertClassifier(nn.Module): #BERT 분류기 클래스

    def __init__(self, dropout=0.5): #클래스 초기화

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased') #HuggingFace에서 미리 학습된 BERT 불러오기 (똑같이 bert-base-cased 이므로 대소문자를 구분)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask): #mask: padding은 무시하라고 알려주는 역할

        #pooled_output : 문장 전체를 이해하고 마지막에 CLS 토큰 위치에 “문장 전체 의미”를 요약한 벡터를 출력
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output) #벡터에 과적합 방지용 dropout 적용
        linear_output = self.linear(dropout_output) #768차원 벡터 -> 5개 클래스로 분류
        final_layer = self.relu(linear_output) #ReLU -> 음수 제거

        return final_layer

In [18]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    #data loader 적용 -> batch
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss() #분류문제이므로
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            # 총 정확도와 손실 누적 변수 초기화
            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader): #배치 단위로 학습

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device) #squeeze(1): 차원 줄이기 ([batch, 1, seq_len] -> [batch, seq_len])

                output = model(input_id, mask) #BERT에 넣어서 예측값 얻기

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item() #가장 높은 확률의 클래스를 예측 결과로 선택해서 정답과 비교
                total_acc_train += acc

                model.zero_grad() #이전 gradient 초기화
                batch_loss.backward() #역전파
                optimizer.step() #파라미터 업데이트

            #검증단계
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            #한 에폭 끝날 때마다 학습 손실/정확도, 검증 손실/정확도 출력
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [13]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad(): #평가 시에는 gradient를 계산할 필요 없음

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    #테스트 정확도
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [14]:
np.random.seed(112)
#데이터 분할. 이때 80%가 df_train, 다음 10% 가 df_val, 마지막 10가 df_test
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test)) #각 세트에 몇 개의 데이터가 있는지 출력

1780 222 223


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [15]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

  0%|          | 0/890 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 890/890 [03:05<00:00,  4.81it/s]


Epochs: 1 | Train Loss:  0.773 | Train Accuracy:  0.323 | Val Loss:  0.672 | Val Accuracy:  0.509


100%|██████████| 890/890 [03:08<00:00,  4.71it/s]


Epochs: 2 | Train Loss:  0.427 | Train Accuracy:  0.803 | Val Loss:  0.220 | Val Accuracy:  0.968


In [16]:
evaluate(model, df_test)

Test Accuracy:  0.982


optional) 다양한 시도를 해보셨다면 시도 별 간단한 해석도 달아주세요! 🤗